In [149]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns 

In [150]:
v1 = pd.read_csv("integrated2.csv")
for i in v1.columns:
    print(i)

Unnamed: 0
lat
lon
PSurf1
PSurf10
PSurf11
PSurf12
PSurf13
PSurf14
PSurf15
PSurf16
PSurf17
PSurf18
PSurf19
PSurf2
PSurf20
PSurf21
PSurf22
PSurf23
PSurf24
PSurf25
PSurf26
PSurf27
PSurf28
PSurf29
PSurf3
PSurf30
PSurf31
PSurf32
PSurf33
PSurf34
PSurf35
PSurf36
PSurf37
PSurf38
PSurf39
PSurf4
PSurf40
PSurf41
PSurf42
PSurf43
PSurf44
PSurf45
PSurf46
PSurf47
PSurf48
PSurf49
PSurf5
PSurf50
PSurf51
PSurf52
PSurf6
PSurf7
PSurf8
PSurf9
Qair1
Qair10
Qair11
Qair12
Qair13
Qair14
Qair15
Qair16
Qair17
Qair18
Qair19
Qair2
Qair20
Qair21
Qair22
Qair23
Qair24
Qair25
Qair26
Qair27
Qair28
Qair29
Qair3
Qair30
Qair31
Qair32
Qair33
Qair34
Qair35
Qair36
Qair37
Qair38
Qair39
Qair4
Qair40
Qair41
Qair42
Qair43
Qair44
Qair45
Qair46
Qair47
Qair48
Qair49
Qair5
Qair50
Qair51
Qair52
Qair6
Qair7
Qair8
Qair9
Rainf1
Rainf10
Rainf11
Rainf12
Rainf13
Rainf14
Rainf15
Rainf16
Rainf17
Rainf18
Rainf19
Rainf2
Rainf20
Rainf21
Rainf22
Rainf23
Rainf24
Rainf25
Rainf26
Rainf27
Rainf28
Rainf29
Rainf3
Rainf30
Rainf31
Rainf32
Rainf33
Rainf3

In [151]:
v1 = v1.drop(columns=['Unnamed: 0' , "geometry" , "CNT_FULLNAME"])

Since the goal is to build clusters and not extracting information from the dataset  we can remove the high correlated data from the dataframe like subSoil / topSoil correlated data 

In [152]:
v1 = v1.drop(columns=['sand % subsoil','silt% subsoil', 'clay % subsoil', 'pH water subsoil','N % subsoil',
        'BS % subsoil','CEC subsoil', 'CEC Clay subsoil','CaCO3 % subsoil',  'BD subsoil'])

We also drop the correlated top soil chimical components

In [153]:
v1 = v1.drop(columns= ['BS % topsoil','BD topsoil'])

In [154]:
v1

,lat,lon,PSurf1,PSurf10,PSurf11,PSurf12,PSurf13,PSurf14,PSurf15,PSurf16,...,clay % topsoil,pH water topsoil,OC % topsoil,OC % subsoil,N % topsoil,CEC topsoil,CEC clay topsoil,CaCO3 % topsoil,C/N topsoil,C/N subsoil
0,-8.25,27.25,89931.291146,89935.560190,89857.437607,89719.336024,89919.630089,89795.942393,89822.866530,89845.084958,...,22.11,7.22,0.583,0.336,0.063,12.58,57.1,5.46,9.8,9.9
1,-8.25,27.75,88198.302755,88202.255054,88144.626482,88011.359357,88205.834726,88097.959524,88101.279536,88143.110607,...,19.09,7.87,0.501,0.288,0.060,9.98,57.8,18.23,10.3,10.8
2,-8.25,28.25,89235.961901,89234.871446,89175.960774,89039.490220,89237.955708,89145.462488,89148.886125,89189.423988,...,19.09,7.87,0.501,0.288,0.060,9.98,57.8,18.23,10.3,10.8
3,-8.25,28.75,89689.187172,89694.181381,89638.232685,89500.870935,89700.801429,89613.034661,89615.432875,89656.726643,...,24.58,7.39,0.678,0.405,0.113,10.87,56.7,8.54,10.3,10.8
4,-7.75,27.25,93016.242297,93018.701482,92912.807863,92766.109494,92984.016548,92840.870970,92899.681970,92893.402655,...,0.00,0.00,0.000,0.000,0.000,0.00,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,10.75,24.25,95072.465089,94959.076833,94828.393851,94597.915542,94820.357179,94638.193423,94533.842458,94701.787935,...,0.00,0.00,0.000,0.000,0.000,0.00,0.0,0.00,0.0,0.0
585,11.25,23.25,94380.135922,94266.675756,94140.533315,93925.996935,94145.756238,93967.888226,93894.973268,94048.788238,...,12.45,3.55,0.485,0.200,0.065,5.20,27.5,0.05,5.5,4.0
586,11.25,23.75,94442.190333,94333.525786,94204.438512,93982.367929,94202.245667,94021.829244,93923.329673,94090.795161,...,0.00,0.00,0.000,0.000,0.000,0.00,0.0,0.00,0.0,0.0
587,11.25,24.25,94815.380714,94705.219577,94574.246238,94340.447714,94563.048720,94380.986940,94259.847762,94441.106321,...,24.80,7.70,0.330,0.230,0.100,8.70,66.0,13.10,8.0,11.0


Normlize the data for better distance calculation

In [155]:
# usin the previous part normalization function 
def MinMax_Normalization(df):
    df_minmax = pd.DataFrame()
    for column in df.columns:
        if df[column].dtype == 'float64' and column not in ["lat","lon"]:
            bas = min(df[column])
            haut = max(df[column])
            gap = haut - bas
            df_minmax[column] = (df[column] - bas)/gap
    return df_minmax

v1 = MinMax_Normalization(v1)
v1

C:\Users\abdo7\AppData\Local\Temp\ipykernel_22024\1251721374.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_minmax[column] = (df[column] - bas)/gap
C:\Users\abdo7\AppData\Local\Temp\ipykernel_22024\1251721374.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_minmax[column] = (df[column] - bas)/gap
C:\Users\abdo7\AppData\Local\Temp\ipykernel_22024\1251721374.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider

,PSurf1,PSurf10,PSurf11,PSurf12,PSurf13,PSurf14,PSurf15,PSurf16,PSurf17,PSurf18,...,clay % topsoil,pH water topsoil,OC % topsoil,OC % subsoil,N % topsoil,CEC topsoil,CEC clay topsoil,CaCO3 % topsoil,C/N topsoil,C/N subsoil
0,0.643969,0.650159,0.649107,0.650683,0.648146,0.649672,0.648948,0.647550,0.647135,0.647381,...,0.366788,0.749740,0.241808,0.452830,0.217241,0.318642,0.758300,0.162791,0.471154,0.61875
1,0.560761,0.566320,0.565992,0.567405,0.565297,0.567261,0.565956,0.565361,0.564443,0.565555,...,0.316689,0.817238,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500
2,0.610584,0.616267,0.616038,0.617535,0.615192,0.618101,0.616457,0.615888,0.614997,0.616900,...,0.316689,0.817238,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500
3,0.632345,0.638483,0.638470,0.640031,0.637567,0.640795,0.638948,0.638454,0.637617,0.640012,...,0.407764,0.767394,0.281211,0.545822,0.389655,0.275329,0.752988,0.254621,0.495192,0.67500
4,0.792091,0.799289,0.797369,0.799240,0.796287,0.797456,0.797271,0.794755,0.795127,0.795795,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,0.890818,0.893144,0.890322,0.888556,0.885061,0.884688,0.876048,0.882083,0.881109,0.879824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
585,0.857577,0.859653,0.856944,0.855794,0.852449,0.852155,0.845250,0.850549,0.848879,0.849049,...,0.206536,0.368640,0.201161,0.269542,0.224138,0.131712,0.365206,0.001491,0.264423,0.25000
586,0.860556,0.862886,0.860045,0.858543,0.855180,0.854773,0.846617,0.852578,0.851415,0.850457,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
587,0.878475,0.880865,0.877990,0.876002,0.872622,0.872205,0.862840,0.869495,0.868750,0.866518,...,0.411413,0.799585,0.136873,0.309973,0.344828,0.220365,0.876494,0.390578,0.384615,0.68750


In [156]:
v1.value_counts()

PSurf1    PSurf10   PSurf11   PSurf12   PSurf13   PSurf14   PSurf15   PSurf16   PSurf17   PSurf18   PSurf19   PSurf2    PSurf20   PSurf21   PSurf22   PSurf23   PSurf24   PSurf25   PSurf26   PSurf27   PSurf28   PSurf29   PSurf3    PSurf30   PSurf31   PSurf32   PSurf33   PSurf34   PSurf35   PSurf36   PSurf37   PSurf38   PSurf39   PSurf4    PSurf40   PSurf41   PSurf42   PSurf43   PSurf44   PSurf45   PSurf46   PSurf47   PSurf48   PSurf49   PSurf5    PSurf50   PSurf51   PSurf52   PSurf6    PSurf7    PSurf8    PSurf9    Qair1     Qair10    Qair11    Qair12    Qair13    Qair14    Qair15    Qair16    Qair17    Qair18    Qair19    Qair2     Qair20    Qair21    Qair22    Qair23    Qair24    Qair25    Qair26    Qair27    Qair28    Qair29    Qair3     Qair30    Qair31    Qair32    Qair33    Qair34    Qair35    Qair36    Qair37    Qair38    Qair39    Qair4     Qair40    Qair41    Qair42    Qair43    Qair44    Qair45    Qair46    Qair47    Qair48    Qair49    Qair5     Qair50    Qair51    Qair52    

In [157]:
v1  = v1.drop_duplicates()
v1

,PSurf1,PSurf10,PSurf11,PSurf12,PSurf13,PSurf14,PSurf15,PSurf16,PSurf17,PSurf18,...,clay % topsoil,pH water topsoil,OC % topsoil,OC % subsoil,N % topsoil,CEC topsoil,CEC clay topsoil,CaCO3 % topsoil,C/N topsoil,C/N subsoil
0,0.643969,0.650159,0.649107,0.650683,0.648146,0.649672,0.648948,0.647550,0.647135,0.647381,...,0.366788,0.749740,0.241808,0.452830,0.217241,0.318642,0.758300,0.162791,0.471154,0.61875
1,0.560761,0.566320,0.565992,0.567405,0.565297,0.567261,0.565956,0.565361,0.564443,0.565555,...,0.316689,0.817238,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500
2,0.610584,0.616267,0.616038,0.617535,0.615192,0.618101,0.616457,0.615888,0.614997,0.616900,...,0.316689,0.817238,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500
3,0.632345,0.638483,0.638470,0.640031,0.637567,0.640795,0.638948,0.638454,0.637617,0.640012,...,0.407764,0.767394,0.281211,0.545822,0.389655,0.275329,0.752988,0.254621,0.495192,0.67500
4,0.792091,0.799289,0.797369,0.799240,0.796287,0.797456,0.797271,0.794755,0.795127,0.795795,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,0.890818,0.893144,0.890322,0.888556,0.885061,0.884688,0.876048,0.882083,0.881109,0.879824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
585,0.857577,0.859653,0.856944,0.855794,0.852449,0.852155,0.845250,0.850549,0.848879,0.849049,...,0.206536,0.368640,0.201161,0.269542,0.224138,0.131712,0.365206,0.001491,0.264423,0.25000
586,0.860556,0.862886,0.860045,0.858543,0.855180,0.854773,0.846617,0.852578,0.851415,0.850457,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
587,0.878475,0.880865,0.877990,0.876002,0.872622,0.872205,0.862840,0.869495,0.868750,0.866518,...,0.411413,0.799585,0.136873,0.309973,0.344828,0.220365,0.876494,0.390578,0.384615,0.68750


Since none of the algorithme we will use is influenced by the presence of outliers we w'ont handle them

# 2. algorithmes coding

In [158]:
def hamming(arr1, arr2):
    if len(arr1) != len(arr2):
        return np.abs(len(arr1) - len(arr2))
    return np.sum(arr1 != arr2)

def manhatten(atr1 , atr2):
    return np.abs(atr1 - atr2)

def distance(df, index, instance, index_inst):
    dist = 0
    for col in df.columns:
        if col not in ["cluster" , "distance"]:
            if df[col].dtype == "object":
                try:
                    dist += 1
                except Exception as e:
                    print(f"Warning: Hamming distance error for column '{col}': {e}.")
                    dist += np.inf  # or some max distance
            else:
                try:
                    dist += manhatten(df.iloc[index][col], instance.iloc[index_inst][col])
                except Exception as e:
                    print(f"Warning: Distance calculation error for column '{col}': {e}.")
                    dist += np.inf  # or some max distance
    return dist

def affect_clusters(df, medoids):
    clusters = [0] * len(df)
    for i in range(len(df)):
        if i % 10 ==0 : 
            print(i)
        min_distance = float('inf')
        for j in range(len(medoids)):
            dist = distance(df, i, medoids, j)
            if min_distance > dist:
                min_distance = dist
                clusters[i] = j
    return clusters

def global_distances(df, medoids):
    df2 = df.copy()
    for i in range(len(df2)):
        cluster_index = int(df.iloc[i]["cluster"])  # Ensure this is valid for medoids
        if cluster_index < len(medoids):
            distance_value = distance(df2, i, medoids, cluster_index)
            df2.loc[i, "distance"] = distance_value
        else:
            print(f"Warning: cluster index {cluster_index} is out of bounds for medoids.")
    
    df2["cluster"] = df["cluster"]  # Ensure cluster assignments are retained
    return df2

def medoids_change_test(df, max_neighbor, nbr_clusters, medoids):
    for _ in range(max_neighbor): 
        for j in range(nbr_clusters):
            df_test = df[df["cluster"] == j]
            current_distance = df_test["distance"].sum()
            print(f"Cluster {j} ---> Current distance: {current_distance}")

            # Randomly select another medoid
            if not df_test.empty: 
                med = df_test.sample(n=1).reset_index(drop=True)
                new_meds = medoids.copy()
                new_meds = new_meds.reset_index(drop=True)
                new_meds.loc[j] = med.iloc[0]

                # Calculate distances with new medoid
                df_test_updated = global_distances(df_test, new_meds)
                second_distance = df_test_updated["distance"].sum()

                print(f"New medoid for cluster {j}: {new_meds.iloc[j]}")
                print(f"New medoid for cluster li nrmlm tkoun {j}: {med.iloc[0]}")
                print(f"Second distance ---> {second_distance}")

                # Update medoids if the new distance is better
                if second_distance < current_distance: 
                    medoids.loc[j] = new_meds.loc[j]  # Only update the specific medoid
                    print(f"Medoid for cluster {j} updated.")

    return medoids


In [ ]:
def Clarans(v1 , nbr_clusters , max_neighbor , numlocal):
    loop =0 
    min_distance = float("inf")
    global_meds = None
    df = v1.copy()
    while loop < numlocal : 
        try: 
            # ncalculiw the old distance df l9dim ida kan kayen 
            min_distance =  df["distance"].sum()
        except Exception as e :
            pass
        # randomly select medoids 
        medoids = df.sample(n=nbr_clusters)
        print("medoids are set")
        # calculate the distances and affect cluster number 
        df["cluster"] = affect_clusters(df , medoids)
        print("clusters are set")
        
        
        df = global_distances(df , medoids)
        print("gloabl distances added")

        # this is changing the df automatically (balak tghlat tnahiha)
        medoids = medoids_change_test(df , max_neighbor , nbr_clusters , medoids)

        current_distance =  df["distance"].sum()
        if current_distance < min_distance : 
            print("meds changed")
            global_meds = medoids
            final_df = df.copy()
        # re do the process numlocal time 
        print("done")
        loop +=1
    return df , global_meds
result , meds = Clarans(v1 , 3 , 2 , 2)
result = global_distances(result , meds) #Ensure the new distances
result

DB Sacan algorithme

In [45]:
meds

,PSurf1,PSurf10,PSurf11,PSurf12,PSurf13,PSurf14,PSurf15,PSurf16,PSurf17,PSurf18,...,clay % topsoil,pH water topsoil,OC % topsoil,OC % subsoil,N % topsoil,CEC topsoil,CEC clay topsoil,CaCO3 % topsoil,C/N topsoil,C/N subsoil
6,0.737310,0.744071,0.742750,0.744313,0.741495,0.743527,0.742512,0.740912,0.740845,0.742131,...,0.316689,0.817238,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500
284,0.739402,0.744412,0.743062,0.742512,0.745582,0.744780,0.746301,0.745186,0.748666,0.748214,...,0.206702,0.404465,0.100166,0.163073,0.162069,0.167553,0.418327,0.202892,0.206731,0.34375
146,0.479222,0.486153,0.484205,0.483958,0.485269,0.485749,0.485233,0.485081,0.486883,0.484424,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [ ]:
df = global_distances(result , meds)
df

,PSurf1,PSurf10,PSurf11,PSurf12,PSurf13,PSurf14,PSurf15,PSurf16,PSurf17,PSurf18,...,OC % topsoil,OC % subsoil,N % topsoil,CEC topsoil,CEC clay topsoil,CaCO3 % topsoil,C/N topsoil,C/N subsoil,cluster,distance
0,0.643969,0.650159,0.649107,0.650683,0.648146,0.649672,0.648948,0.647550,0.647135,0.647381,...,0.241808,0.452830,0.217241,0.318642,0.758300,0.162791,0.471154,0.61875,0,15.338189
1,0.560761,0.566320,0.565992,0.567405,0.565297,0.567261,0.565956,0.565361,0.564443,0.565555,...,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500,0,27.478504
2,0.610584,0.616267,0.616038,0.617535,0.615192,0.618101,0.616457,0.615888,0.614997,0.616900,...,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500,0,18.090703
3,0.632345,0.638483,0.638470,0.640031,0.637567,0.640795,0.638948,0.638454,0.637617,0.640012,...,0.281211,0.545822,0.389655,0.275329,0.752988,0.254621,0.495192,0.67500,0,18.508888
4,0.792091,0.799289,0.797369,0.799240,0.796287,0.797456,0.797271,0.794755,0.795127,0.795795,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0,19.825149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,0.890818,0.893144,0.890322,0.888556,0.885061,0.884688,0.876048,0.882083,0.881109,0.879824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1,61.285126
585,0.857577,0.859653,0.856944,0.855794,0.852449,0.852155,0.845250,0.850549,0.848879,0.849049,...,0.201161,0.269542,0.224138,0.131712,0.365206,0.001491,0.264423,0.25000,1,53.843527
586,0.860556,0.862886,0.860045,0.858543,0.855180,0.854773,0.846617,0.852578,0.851415,0.850457,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1,58.794008
587,0.878475,0.880865,0.877990,0.876002,0.872622,0.872205,0.862840,0.869495,0.868750,0.866518,...,0.136873,0.309973,0.344828,0.220365,0.876494,0.390578,0.384615,0.68750,1,62.692073


In [58]:
df.iloc[146]

PSurf1              0.479222
PSurf10             0.486153
PSurf11             0.484205
PSurf12             0.483958
PSurf13             0.485269
                     ...    
CaCO3 % topsoil     0.000000
C/N topsoil         0.000000
C/N subsoil         0.000000
cluster             1.000000
distance           54.700986
Name: 146, Length: 274, dtype: float64

In [53]:
df["distance"].value_counts()

distance
0.000000     2
15.338189    1
27.478504    1
18.090703    1
18.508888    1
            ..
61.285126    1
53.843527    1
58.794008    1
62.692073    1
53.992042    1
Name: count, Length: 588, dtype: int64

DB SCAN ALGORITHME

In [207]:
import numpy as np
from scipy.spatial import KDTree

def calculate_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

def define_core_points(v1, max_ngbr, circle_size):
    result=v1.copy()
    df = v1.values  # Assuming v1 is a DataFrame
    all_neighbors = {}
    
    for i in range(len(df)):
        neighbors = []
        for j in range(len(df)):
            if i != j:
                dist = calculate_distance(df[i], df[j])
                if dist <= circle_size:
                    neighbors.append(j)
        
        # Limit to max_ngbr if necessary
        if max_ngbr is not None:
            neighbors = neighbors[:]

        all_neighbors[i] = neighbors
        print(f"Point {i} has {len(neighbors)} neighbors: {neighbors}")
        if len(all_neighbors[i]) > max_ngbr : 
            result.loc[i , "is_core"] = True
        else : 
            result.loc[i , "is_core"] = False
    return all_neighbors , result

def clustering(df , neighbor_matrix):
    nbr_cluster = 0
    clusters = df.copy()
    not_core=[]
    while True:
        try :
            test = clusters[pd.isna(clusters["cluster"])]
            print(len(test))
            if len(test[test["is_core"]==False])==len(test):
                break
        except Exception as e:
            test = clusters.copy()
        # randomly select a core point
        while True:
            point = test.sample(n=1)
            point = point[point["is_core"]==True]
            if len(point) == 1:
                break
        index = df.index[point.iloc[0].name]
        print(index)
        stack = [index]
        # set of core points najoutiw el false point w nkhrjou les voisin ta3 other core points
        while len(stack) > 0:
            index = stack.pop()
            clusters.loc[index, "cluster"] = nbr_cluster
            
            for i in neighbor_matrix[index]:
                if clusters.iloc[i]["is_core"] == True and pd.isna(clusters.iloc[i]["cluster"]):
                    stack.append(i)

                elif clusters.iloc[i]["is_core"] == False and i not in not_core:
                    not_core.append(i)
        for j in not_core:
            clusters.loc[j, "cluster"] = nbr_cluster
        
        nbr_cluster+=1
    return clusters



In [214]:
def dbScan(v1,circle_size , max_ngbr):
    
    # define core points (points li andhom niehgbors kter man max ngbr) , neighbore m3ntha distance < l9otr ta3 da2ira
    neighbor_matrix , df= define_core_points(v1 , max_ngbr , circle_size)
    # nselectioniw core point wahed w ndkhlouh f cluster apres nseclctioniw neighbore ta3ou w njoutiwhom l cluster (lazem ykounou core points )
    df["cluster"] = pd.NA
    df = clustering(df , neighbor_matrix)
    # ki nkmlou ga3 li f cluster wahed nsotiw l non core point li jazna 3lihom w ndkhlohom f hadak el cluster b chart ykounou neihbore ta3 core points man hadak el cluster 
    # le faite li nkmlou n3wdou nkhyrou random core point whdokher w najoutiwah l cluster wahdokher ... etc 
    # repeter juesqu'a ykhlassou ga3 
    return df
df = dbScan(v1 , 1.6 , 4)

Point 0 has 9 neighbors: [1, 2, 3, 6, 7, 12, 19, 20, 26]
Point 1 has 6 neighbors: [0, 2, 3, 7, 13, 20]
Point 2 has 8 neighbors: [0, 1, 3, 6, 7, 13, 20, 26]
Point 3 has 10 neighbors: [0, 1, 2, 6, 7, 13, 20, 26, 27, 34]
Point 4 has 10 neighbors: [5, 9, 10, 11, 15, 16, 17, 22, 24, 31]
Point 5 has 5 neighbors: [4, 9, 10, 11, 17]
Point 6 has 8 neighbors: [0, 2, 3, 7, 12, 18, 19, 33]
Point 7 has 10 neighbors: [0, 1, 2, 3, 6, 8, 13, 20, 26, 34]
Point 8 has 4 neighbors: [7, 13, 14, 21]
Point 9 has 10 neighbors: [4, 5, 10, 11, 15, 16, 17, 22, 24, 31]
Point 10 has 11 neighbors: [4, 5, 9, 11, 15, 16, 17, 22, 23, 24, 31]
Point 11 has 8 neighbors: [4, 5, 9, 10, 16, 17, 22, 24]
Point 12 has 7 neighbors: [0, 6, 15, 18, 19, 25, 33]
Point 13 has 8 neighbors: [1, 2, 3, 7, 8, 20, 21, 26]
Point 14 has 8 neighbors: [8, 21, 55, 65, 75, 76, 77, 86]
Point 15 has 21 neighbors: [4, 9, 10, 12, 16, 17, 18, 19, 22, 23, 24, 25, 28, 29, 30, 31, 32, 33, 38, 39, 40]
Point 16 has 21 neighbors: [4, 9, 10, 11, 15, 17, 18

In [215]:
df["cluster"].value_counts()

cluster
0    486
1     43
Name: count, dtype: int64